In [1]:
import pandas as pd
import numpy as np

final_merged_df = pd.read_csv("VIP_30k_merged_data_59364_822.csv")

# 1. 데이터 정렬 (회원별, 시간순)
df = final_merged_df.sort_values(by=['발급회원번호', '기준년월']).copy()

# 2. 이용금액 합산 및 평균 계산
# 당월 총 이용금액 (신용 + 체크)
df['당월_총_이용금액'] = df['이용금액_신용_B0M'] + df['이용금액_체크_B0M']

# 직전 3개월 평균 이용금액 (신용 R3M + 체크 R3M) / 3
df['직전_3M_평균_이용금액'] = (df['이용금액_신용_R3M'] + df['이용금액_체크_R3M']) / 3

# 3. 이탈 타겟 정의 로직 (Method B)
def define_churn_strict(row):
    # 직전 3개월 평균이 없거나 0원인 경우 -> '판단 제외(휴면/신규)'로 분류
    if pd.isna(row['직전_3M_평균_이용금액']) or row['직전_3M_평균_이용금액'] <= 0:
        return np.nan 
    
    # 직전 평균 대비 10% 미만 사용 시 이탈(1), 아니면 유지(0)
    if row['당월_총_이용금액'] < (row['직전_3M_평균_이용금액'] * 0.1):
        return 1
    else:
        return 0

df['이탈_타겟'] = df.apply(define_churn_strict, axis=1)

# 4. 데이터 분리 (학습용 vs 휴면/신규 제외용)
# 타겟이 0 또는 1로 확실히 정해진 데이터 (학습용)
train_df = df[df['이탈_타겟'].notna()].copy()
train_df['이탈_타겟'] = train_df['이탈_타겟'].astype(int)

# 타겟이 NaN인 데이터 (별도 관리용: 장기 휴면 또는 신규 고객)
dormant_new_df = df[df['이탈_타겟'].isna()].copy()

# 5. 결과 확인
print("--- [데이터 분리 결과] ---")
print(f"1. 학습 가능 데이터(0, 1): {len(train_df)}건")
print(f"   - 유지(0): {len(train_df[train_df['이탈_타겟'] == 0])}건")
print(f"   - 이탈(1): {len(train_df[train_df['이탈_타겟'] == 1])}건")
print(f"\n2. 판단 제외 데이터(NaN): {len(dormant_new_df)}건 (장기 미사용/신규)")
print("--------------------------")

# 6. (선택) 각각 저장
# train_df.to_csv('3M_VIP_Target_data.csv', index=False)
# dormant_new_df.to_csv('dormant_customers.csv', index=False)



###### 추가적인 기울기 ---------------------------------
from sklearn.linear_model import LinearRegression

# 기간별 이용금액의 변화 추세(기울기)를 계산하는 함수 정의
def get_spending_slope(amount_list):
    # 데이터가 부족하여 기울기 산출이 불가능한 경우 0.0 반환
    if len(amount_list) < 2:
        return 0.0
    
    # X축(시간 순서)과 Y축(이용금액) 데이터 구성
    y = np.array(amount_list).reshape(-1, 1)
    x = np.arange(len(y)).reshape(-1, 1)
    
    # 결측치 존재 시 0으로 대체 처리
    y = np.nan_to_num(y)
    
    # 선형 회귀 모델 적합을 통한 기울기 계수 추출
    lr = LinearRegression()
    lr.fit(x, y)
    
    return lr.coef_[0][0]

# 회원별로 시계열 순서에 따른 당월_총_이용금액 리스트 생성
# 상림이의 기존 코드에서 정렬된 df를 활용함
slope_data = df.groupby('발급회원번호')['당월_총_이용금액'].apply(list).reset_index()
slope_data.columns = ['발급회원번호', '이용금액_리스트']

# 각 회원별 이용금액 시퀀스에 대하여 기울기 계산 함수 적용
slope_data['금액_기울기'] = slope_data['이용금액_리스트'].apply(get_spending_slope)

# 계산된 기울기 변수를 기존 학습용 데이터프레임(train_df)에 결합
# 발급회원번호를 기준으로 Left Join 수행
train_df = pd.merge(train_df, slope_data[['발급회원번호', '금액_기울기']], on='발급회원번호', how='left')

# 기울기 계산이 불가능하여 발생한 결측치는 0으로 보정
train_df['금액_기울기'] = train_df['금액_기울기'].fillna(0)

# 최종 데이터셋의 컬럼 구성 및 상위 데이터 확인
print("--- [최종 데이터셋 구성 확인] ---")
print(f"최종 피처 개수: {len(train_df.columns)}개 (기존 피처 + 이탈_타겟 + 금액_기울기)")
print(train_df[['발급회원번호', '이탈_타겟', '금액_기울기']].head())

--- [데이터 분리 결과] ---
1. 학습 가능 데이터(0, 1): 57357건
   - 유지(0): 56727건
   - 이탈(1): 630건

2. 판단 제외 데이터(NaN): 2007건 (장기 미사용/신규)
--------------------------
--- [최종 데이터셋 구성 확인] ---
최종 피처 개수: 826개 (기존 피처 + 이탈_타겟 + 금액_기울기)
       발급회원번호  이탈_타겟        금액_기울기
0  SYN_100022      0  28027.771429
1  SYN_100022      0  28027.771429
2  SYN_100022      0  28027.771429
3  SYN_100022      0  28027.771429
4  SYN_100022      0  28027.771429


In [4]:
train_df['금액_기울기']

0        28027.771429
1        28027.771429
2        28027.771429
3        28027.771429
4        28027.771429
             ...     
57352   -10254.714286
57353   -10254.714286
57354   -10254.714286
57355   -10254.714286
57356   -10254.714286
Name: 금액_기울기, Length: 57357, dtype: float64